In [111]:
# ¿Es activa?
# tine un puente salino y tiene como inhibidor al ATP
import MDAnalysis
ref_id = "1fin"
dfg_out = "4gcj"
pdbs_ENS_dir = "../ARCHIVOS/CRISTALES/PROT_CDK2_PREPARADAS/PREP_ALL_ENSAMBLE/"
ref_structure_file = pdbs_ENS_dir + ref_id + "_ENS.pdb"
ref_dfgOut_file = pdbs_ENS_dir + dfg_out + "_ENS.pdb"

In [129]:
import MDAnalysis
from MDAnalysis.analysis.dihedrals import Dihedral
import MDAnalysis.analysis.hbonds
import numpy as np

lista_res = {}

pdb_id_list = ["4gcj", "1aq1", "4fkw", "3pxr", "1fin", "5a14"]
pdbs_ENS_dir = "../ARCHIVOS/CRISTALES/PROT_CDK2_PREPARADAS/PREP_ALL_ENSAMBLE/"

for pdb_id in pdb_id_list:
    pdb_id_file = pdbs_ENS_dir + pdb_id + "_ENS.pdb"
    u = MDAnalysis.Universe(pdb_id_file)

    # ¿Es activa? o dfgout, será activa si tiene un puente salino entre Lys33 y Glu51 y no es DFGOUT
    hb_K33_Q51 = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, 
                            'resnum 33', 'resnum 51', distance=3.0, angle=120.0)
    hb_K33_Q51.run()
    hb_K33_Q51.generate_table()
    if len(hb_K33_Q51.table) > 0:
        # Es DFG-out
        phi_res_D145 = [ u.residues[144].phi_selection() ]
        phi_value = ( Dihedral(phi_res_D145).run() ).angles[0][0]
        if phi_value < 0:
            print(pdb_id, "Es dfg-out")
            d = {pdb_id: "dfg_out"}
        else:
            print(pdb_id, "Es activa")
            d = {pdb_id: "active"}
    # Entonces es abierta, inhibidora A o B
    else:
        aC = u.select_atoms("resnum 46-55").center_of_geometry()
        B4 = u.select_atoms("resnum 76-81").center_of_geometry()
        dist_aC_B4 = np.linalg.norm(B4 - aC)
        # ¿Es abierta?
        if dist_aC_B4 > 14.5:
            print(pdb_id, "Es abierta")
            d = {pdb_id: "open"}
        else:
            hb_helix_ALoop = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, 
                             'resnum 147-155', 'resnum 147-155', distance=3.0, angle=120.0)
            hb_helix_ALoop.run()
            hb_helix_ALoop.generate_table()
             # Entonces es inicativa A?
            if len( hb_helix_ALoop.table ) > 1:
                print(pdb_id, "Es Inactiva A")
                d = {pdb_id: "inactive_A"}
            else:
                print(pdb_id, "Es Inactiva B")
                d = {pdb_id: "inactive_B"}
    lista_res.update(d)

pd.Series(lista_res)

4gcj Es Inactiva B
1aq1 Es Inactiva B
4fkw Es Inactiva B
3pxr Es Inactiva A
1fin Es activa
5a14 Es dfg-out


4gcj    inactive_B
1aq1    inactive_B
4fkw    inactive_B
3pxr    inactive_A
1fin        active
5a14       dfg_out
dtype: object

In [94]:
u = MDAnalysis.Universe(ref_structure_file)
g = MDAnalysis.Universe(ref_dfgOut_file)
ags_1 = [res.phi_selection() for res in u.residues[144:145]]
ags_2 = [res.phi_selection() for res in g.residues[144:145]]

from MDAnalysis.analysis.dihedrals import Dihedral
r = Dihedral(ags_1).run()
s = Dihedral(ags_2).run()
print(r.angles, s.angles)

u.residues[144].phi_selection()
ags_1

[[60.17351814]] [[57.6460866]]


[<AtomGroup with 4 atoms>]

In [87]:
# PAra comprobar si es Inactivo A o B
import MDAnalysis
import MDAnalysis.analysis.hbonds

h = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, 'resnum 147-155', 'resnum 147-155', distance=3.0, angle=120.0)
h.run()
h.generate_table()
import pandas as pd
len(h.table)

3

In [95]:
# PAra comprobar si es Inactivo A o B
import MDAnalysis
import MDAnalysis.analysis.hbonds

h = MDAnalysis.analysis.hbonds.HydrogenBondAnalysis(u, 'resnum 33', 'resnum 51', distance=3.0, angle=120.0)
h.run()
h.generate_table()
len(h.table)

1

In [116]:
# tine un puente salino y tiene como inhibidor al ATP
import MDAnalysis
ref_id = "1aq1"
dfg_out = "1aq1"
pdbs_ENS_dir = "../ARCHIVOS/CRISTALES/PROT_CDK2_PREPARADAS/PREP_ALL_ENSAMBLE/"
ref_structure_file = pdbs_ENS_dir + ref_id + "_ENS.pdb"
ref_dfgOut_file = pdbs_ENS_dir + dfg_out + "_ENS.pdb"

u = MDAnalysis.Universe(ref_structure_file)
g = MDAnalysis.Universe(ref_dfgOut_file)

# Para comprobar si es abierto
import numpy as np
aC = g.select_atoms("resnum 46-55").center_of_geometry()
b4 = g.select_atoms("resnum 76-81").center_of_geometry()

print(np.linalg.norm(b4 - aC) > 14)
#print(aC, b4)

False
